In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')
"""
Function GRS_test(factor, resid, alpha) is to conduct GRS test according 
to Gibbons, Ross & Shanken(1989) to receive GRS-statistic and p-value.

H0: alpha1=alpha2=...=alphaN

Parameters:
  T = number of months
  N = number of portfolios
  L = number of factors

Inputs:
  factor: matrix of FF factors with shape (T, L)
  resid: matrix of residuals with shape (T, N)
  alpha: matrix of intercepts with shape (N, 1)

Outputs:
  f_grs: GRS-statistic
  p_grs: P-value

"""

def GRS_test(factor, resid, alpha):
    N = resid.shape[1]        
    T = resid.shape[0]       
    L = factor.shape[1]      

    if (T-N-L) < 0:
        print('can not conduct GRS test because T-N-L<0')
        return

    factor = np.asmatrix(factor)                   # factor matrix (T, L)
    resid = np.asmatrix(resid)                     # residual matrix (T, N)
    alpha = np.asmatrix(alpha).reshape(N, 1)       # intercept matrix (N, 1)

    mean_return_factor = (factor.mean(axis=0))

    # covariance matrix of residuals
    cov_resid = (resid.T * resid) / (T-L-1)
    # covariance matrix of factors
    cov_factor = ((factor - mean_return_factor).T * (factor - mean_return_factor)) / (T-1)

    mean_return_factor = mean_return_factor.reshape(L, 1)

    # GRS statistic
    f_grs = float((T/N) * ((T-N-L)/(T-L-1)) * ((alpha.T * np.linalg.inv(cov_resid) * alpha) / (1 + mean_return_factor.T * np.linalg.inv(cov_factor) * mean_return_factor)))

    # p-value
    p_grs = 1 - st.f.cdf(f_grs, N, (T-N-L))

    return f_grs, p_grs

In [2]:
factor = pd.concat([pd.read_csv('factor_result/2016FF5.csv'),pd.read_csv('factor_result/2017FF5.csv')], axis= 0)
factor.set_index('date', inplace= True)

In [3]:
BM_resid = pd.read_csv('regression_result/regression_BM_resid.csv')
BM_alpha = pd.read_csv('regression_result/regression_BM.csv')
BM_alpha = BM_alpha['Const']
GRS_test(factor, BM_resid, BM_alpha)

(5.306214040619461, 0.040383604177044785)

In [9]:
BM_a = np.abs(BM_alpha).mean()
BM_a

0.004064312405166923

In [4]:
Inv_resid = pd.read_csv('regression_result/regression_Inv_resid.csv')
Inv_alpha = pd.read_csv('regression_result/regression_Inv.csv')
Inv_alpha = Inv_alpha['Const']
GRS_test(factor, Inv_resid, Inv_alpha)

(3.9366705310226133, 0.07274418058882959)

In [14]:
Inv_a = np.abs(Inv_alpha)**2
Inv_a.mean()

1.3272762004543406e-05

In [5]:
OP_resid = pd.read_csv('regression_result/regression_OP_resid.csv')
OP_alpha = pd.read_csv('regression_result/regression_OP.csv')
OP_alpha = OP_alpha['Const']
GRS_test(factor, OP_resid, OP_alpha)

(7.515798429076026, 0.019426173430787452)

In [11]:
OP_a = np.abs(OP_alpha).mean()
OP_a

0.004101142442207568